In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("C:/Users/anura_000/Downloads/twitter_airline.csv")
test = pd.read_csv("C:/Users/anura_000/Downloads/twitter_test.csv")

In [3]:
train.head(2)

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)


In [4]:
train.count()

tweet_id                  10980
airline_sentiment         10980
airline                   10980
airline_sentiment_gold       31
name                      10980
negativereason_gold          24
retweet_count             10980
text                      10980
tweet_coord                 776
tweet_created             10980
tweet_location             7430
user_timezone              7403
dtype: int64

In [5]:
# Dropping Useless Columns

train.drop('tweet_id', inplace = True, axis = 1)
train.drop('airline_sentiment_gold', inplace = True, axis = 1)
train.drop('negativereason_gold', inplace = True, axis = 1)
train.drop('tweet_coord', inplace = True, axis = 1)
train.drop('tweet_created',inplace = True, axis = 1)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10980 entries, 0 to 10979
Data columns (total 7 columns):
airline_sentiment    10980 non-null object
airline              10980 non-null object
name                 10980 non-null object
retweet_count        10980 non-null int64
text                 10980 non-null object
tweet_location       7430 non-null object
user_timezone        7403 non-null object
dtypes: int64(1), object(6)
memory usage: 600.5+ KB


In [7]:
train.head(2)

,airline_sentiment,airline,name,retweet_count,text,tweet_location,user_timezone
0,negative,Southwest,ColeyGirouard,0,"@SouthwestAir I am scheduled for the morning, ...",Washington D.C.,Atlantic Time (Canada)
1,positive,Southwest,WalterFaddoul,0,@SouthwestAir seeing your workers time in and ...,"Indianapolis, Indiana; USA",Central Time (US & Canada)


### Filling NaN values

In [8]:
mood_count = train['airline_sentiment'].value_counts()
mood_count

negative    6851
neutral     2327
positive    1802
Name: airline_sentiment, dtype: int64

In [9]:
train['airline'].value_counts()

United            2928
US Airways        2152
American          2078
Southwest         1817
Delta             1639
Virgin America     366
Name: airline, dtype: int64

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['airline_sentiment'] = le.fit_transform(train['airline_sentiment'])
train.head(2)

,airline_sentiment,airline,name,retweet_count,text,tweet_location,user_timezone
0,0,Southwest,ColeyGirouard,0,"@SouthwestAir I am scheduled for the morning, ...",Washington D.C.,Atlantic Time (Canada)
1,2,Southwest,WalterFaddoul,0,@SouthwestAir seeing your workers time in and ...,"Indianapolis, Indiana; USA",Central Time (US & Canada)


In [11]:
import re
import nltk
from nltk.corpus import stopwords

In [12]:
def tweet_to_words(raw_tweet):
    letters_only = re.sub("[^a-zA-Z]", " ",raw_tweet) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops] 
    return( " ".join( meaningful_words )) 

In [13]:
train['clean_tweet'] = train['text'].apply(lambda x: tweet_to_words(x))

test['clean_tweet'] = test['text'].apply(lambda x: tweet_to_words(x))

In [14]:
train_clean_tweet=[]
for tweet in train['clean_tweet']:
    train_clean_tweet.append(tweet)
    
test_clean_tweet=[]
for tweet in test['clean_tweet']:
    test_clean_tweet.append(tweet)

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(analyzer = "word")
train_features= v.fit_transform(train_clean_tweet)
test_features= v.transform(test_clean_tweet)

In [27]:
train_features

<10980x11471 sparse matrix of type '<class 'numpy.int64'>'
	with 107004 stored elements in Compressed Sparse Row format>

In [16]:
from sklearn.naive_bayes import MultinomialNB

In [17]:
nb = MultinomialNB()

In [28]:
nb.fit(train_features,train['airline_sentiment'])
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy = accuracy_score(nb.predict(train_features),train['airline_sentiment'])
print(accuracy)

0.8437158469945355


In [30]:
test_features

<3660x11471 sparse matrix of type '<class 'numpy.int64'>'
	with 33563 stored elements in Compressed Sparse Row format>

In [31]:
test_pred

array(['negative', 'negative', 'negative', ..., 'neutral', 'positive',
       'neutral'], dtype=object)

In [19]:
test_pred = nb.predict(test_features)
test_pred = le.inverse_transform(test_pred)
print(test_pred)
np.savetxt("test_pred.csv",test_pred,fmt='%s')

['negative' 'negative' 'negative' ... 'neutral' 'positive' 'neutral']


C:\Users\anura_000\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [20]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
clf = RandomForestClassifier(n_estimators = 200)

In [25]:
train_features

<10980x11471 sparse matrix of type '<class 'numpy.int64'>'
	with 107004 stored elements in Compressed Sparse Row format>

In [22]:
clf.fit(train_features,train['airline_sentiment'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy = accuracy_score(clf.predict(train_features),train['airline_sentiment'])
print(accuracy)

In [ ]:
test_pred = clf.predict(test_features)

In [ ]:
test_pred = le.inverse_transform(test_pred)

In [ ]:
np.savetxt("test_pred.csv",test_pred,fmt='%s')

In [ ]:
print(test_pred)